In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
%load_ext tensorboard

In [2]:
'''
import glob
import pickle
files = glob.glob("../../datasets/ant-*.csv", recursive = True)
globa = pd.concat(map(pd.read_csv, files))
pickle.dump( globa, open( "ant_master.p", "wb" ) )
'''

'\nimport glob\nimport pickle\nfiles = glob.glob("../../datasets/ant-*.csv", recursive = True)\ngloba = pd.concat(map(pd.read_csv, files))\npickle.dump( globa, open( "ant_master.p", "wb" ) )\n'

In [3]:
ant_master = pickle.load( open( "poi_master.p", "rb" ) )

In [4]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,poi,2.5.1,org.apache.poi.hssf.record.ValueRangeRecord,40,2,0,6,60,90,1,...,0.937500,9,0.229167,0.275000,1,6,18.250000,2,0.9500,2
1,poi,2.5.1,org.apache.poi.ddf.EscherRecord,20,1,14,32,36,164,31,...,1.000000,0,0.000000,0.237500,0,0,5.950000,2,1.0500,0
2,poi,2.5.1,org.apache.poi.hssf.usermodel.HSSFRow,24,1,0,9,52,86,4,...,0.833333,3,0.000000,0.183333,0,0,17.541667,5,1.7083,4
3,poi,2.5.1,org.apache.poi.hssf.record.ObjectLinkRecord,16,2,0,5,28,72,1,...,0.333333,0,0.458333,0.468750,1,6,14.312500,2,0.8750,2
4,poi,2.5.1,org.apache.poi.hssf.record.formula.NamePtg,8,2,0,5,20,4,2,...,0.600000,0,0.583333,0.375000,1,2,16.875000,1,0.6250,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,poi,2.0RC1,org.apache.poi.hssf.record.PrintHeadersRecord,12,2,0,5,22,46,2,...,0.500000,0,0.550000,0.400000,1,6,9.500000,2,1.0000,0
310,poi,2.0RC1,org.apache.poi.util.ShortList,25,1,0,0,29,0,0,...,1.000000,0,0.000000,0.313333,1,1,24.040000,9,2.6000,0
311,poi,2.0RC1,org.apache.poi.hssf.usermodel.HSSFFooter,16,1,0,2,30,88,1,...,0.000000,1,0.000000,0.218750,0,0,14.375000,4,1.1250,0
312,poi,2.0RC1,org.apache.poi.hssf.record.HCenterRecord,12,2,0,6,22,46,3,...,0.500000,0,0.550000,0.400000,1,6,9.500000,2,1.0000,0


In [6]:
test = ant_master[ant_master['version']==3]
test

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,poi,3,org.apache.poi.poifs.storage.HeaderBlockReader,7,1,0,8,21,9,2,...,1.000000,5,0.000000,0.571429,0,0,27.714286,2,1.0000,1
1,poi,3,org.apache.poi.hssf.record.formula.RefVPtg,5,3,0,4,13,10,2,...,0.000000,0,0.920000,0.400000,1,1,6.800000,1,0.4000,0
2,poi,3,org.apache.poi.hssf.usermodel.HSSFChildAnchor,5,2,0,5,7,6,4,...,0.000000,0,0.769231,0.700000,1,1,8.200000,2,1.0000,1
3,poi,3,org.apache.poi.hssf.record.LeftMarginRecord,11,2,0,7,21,35,2,...,0.500000,0,0.550000,0.257576,1,4,8.545455,2,0.9091,1
4,poi,3,org.apache.poi.hssf.record.formula.SubtractPtg,9,2,0,5,13,36,2,...,0.000000,0,0.300000,0.259259,0,0,5.333333,1,0.7778,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,poi,3,org.apache.poi.poifs.storage.BlockListImpl,6,1,2,5,13,0,2,...,1.000000,2,0.000000,0.458333,0,0,16.500000,3,1.1667,3
438,poi,3,org.apache.poi.hpsf.wellknown.PropertyIDMap,7,3,0,1,18,21,1,...,0.052632,2,0.915254,0.224490,1,2,36.571429,2,1.0000,0
439,poi,3,org.apache.poi.hssf.record.formula.ReferencePtg,21,2,5,14,42,38,8,...,0.857143,3,0.291667,0.196429,1,2,13.809524,3,0.9048,1
440,poi,3,org.apache.poi.hssf.record.TabIdRecord,10,2,0,6,24,33,2,...,0.000000,0,0.578947,0.266667,1,4,15.400000,2,1.2000,1


In [7]:
train = ant_master[ant_master['version']!=3]
train

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,poi,2.5.1,org.apache.poi.hssf.record.ValueRangeRecord,40,2,0,6,60,90,1,...,0.937500,9,0.229167,0.275000,1,6,18.250000,2,0.9500,2
1,poi,2.5.1,org.apache.poi.ddf.EscherRecord,20,1,14,32,36,164,31,...,1.000000,0,0.000000,0.237500,0,0,5.950000,2,1.0500,0
2,poi,2.5.1,org.apache.poi.hssf.usermodel.HSSFRow,24,1,0,9,52,86,4,...,0.833333,3,0.000000,0.183333,0,0,17.541667,5,1.7083,4
3,poi,2.5.1,org.apache.poi.hssf.record.ObjectLinkRecord,16,2,0,5,28,72,1,...,0.333333,0,0.458333,0.468750,1,6,14.312500,2,0.8750,2
4,poi,2.5.1,org.apache.poi.hssf.record.formula.NamePtg,8,2,0,5,20,4,2,...,0.600000,0,0.583333,0.375000,1,2,16.875000,1,0.6250,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,poi,2.0RC1,org.apache.poi.hssf.record.PrintHeadersRecord,12,2,0,5,22,46,2,...,0.500000,0,0.550000,0.400000,1,6,9.500000,2,1.0000,0
310,poi,2.0RC1,org.apache.poi.util.ShortList,25,1,0,0,29,0,0,...,1.000000,0,0.000000,0.313333,1,1,24.040000,9,2.6000,0
311,poi,2.0RC1,org.apache.poi.hssf.usermodel.HSSFFooter,16,1,0,2,30,88,1,...,0.000000,1,0.000000,0.218750,0,0,14.375000,4,1.1250,0
312,poi,2.0RC1,org.apache.poi.hssf.record.HCenterRecord,12,2,0,6,22,46,3,...,0.500000,0,0.550000,0.400000,1,6,9.500000,2,1.0000,0


In [8]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [9]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,40,2,0,6,60,90,1,5,38,0.838141,786,0.937500,9,0.229167,0.275000,1,6,18.250000,2,0.9500
1,20,1,14,32,36,164,31,4,18,0.842105,141,1.000000,0,0.000000,0.237500,0,0,5.950000,2,1.0500
2,24,1,0,9,52,86,4,5,16,0.673913,451,0.833333,3,0.000000,0.183333,0,0,17.541667,5,1.7083
3,16,2,0,5,28,72,1,4,14,0.933333,254,0.333333,0,0.458333,0.468750,1,6,14.312500,2,0.8750
4,8,2,0,5,20,4,2,4,7,0.800000,148,0.600000,0,0.583333,0.375000,1,2,16.875000,1,0.6250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,12,2,0,5,22,46,2,3,10,0.818182,128,0.500000,0,0.550000,0.400000,1,6,9.500000,2,1.0000
310,25,1,0,0,29,0,0,0,24,0.388889,629,1.000000,0,0.000000,0.313333,1,1,24.040000,9,2.6000
311,16,1,0,2,30,88,1,1,14,0.683333,250,0.000000,1,0.000000,0.218750,0,0,14.375000,4,1.1250
312,12,2,0,6,22,46,3,3,10,0.818182,128,0.500000,0,0.550000,0.400000,1,6,9.500000,2,1.0000


In [10]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,7,1,0,8,21,9,2,6,7,0.833333,207,1.000000,5,0.000000,0.571429,0,0,27.714286,2,1.0000
1,5,3,0,4,13,10,2,2,4,1.250000,40,0.000000,0,0.920000,0.400000,1,1,6.800000,1,0.4000
2,5,2,0,5,7,6,4,1,5,2.000000,46,0.000000,0,0.769231,0.700000,1,1,8.200000,2,1.0000
3,11,2,0,7,21,35,2,5,9,0.800000,107,0.500000,0,0.550000,0.257576,1,4,8.545455,2,0.9091
4,9,2,0,5,13,36,2,3,9,1.125000,59,0.000000,0,0.300000,0.259259,0,0,5.333333,1,0.7778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,6,1,2,5,13,0,2,3,4,0.500000,107,1.000000,2,0.000000,0.458333,0,0,16.500000,3,1.1667
438,7,3,0,1,18,21,1,0,7,1.149123,301,0.052632,2,0.915254,0.224490,1,2,36.571429,2,1.0000
439,21,2,5,14,42,38,8,7,20,0.685714,318,0.857143,3,0.291667,0.196429,1,2,13.809524,3,0.9048
440,10,2,0,6,24,33,2,4,8,0.777778,166,0.000000,0,0.578947,0.266667,1,4,15.400000,2,1.2000


In [11]:
y_train = train['bug']
y_test = test['bug']

In [12]:
y_train

0      2
1      0
2      4
3      2
4      1
      ..
309    0
310    0
311    0
312    0
313    0
Name: bug, Length: 936, dtype: int64

In [13]:
y_test

0      1
1      0
2      1
3      1
4      0
      ..
437    3
438    0
439    1
440    1
441    0
Name: bug, Length: 442, dtype: int64

In [14]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [15]:
x_train_np

array([[40.        ,  2.        ,  0.        , ..., 18.25      ,
         2.        ,  0.95      ],
       [20.        ,  1.        , 14.        , ...,  5.95      ,
         2.        ,  1.05      ],
       [24.        ,  1.        ,  0.        , ..., 17.54166667,
         5.        ,  1.7083    ],
       ...,
       [16.        ,  1.        ,  0.        , ..., 14.375     ,
         4.        ,  1.125     ],
       [12.        ,  2.        ,  0.        , ...,  9.5       ,
         2.        ,  1.        ],
       [25.        ,  1.        ,  0.        , ..., 24.04      ,
         9.        ,  2.6       ]])

In [16]:
x_test_np

array([[ 7.        ,  1.        ,  0.        , ..., 27.71428571,
         2.        ,  1.        ],
       [ 5.        ,  3.        ,  0.        , ...,  6.8       ,
         1.        ,  0.4       ],
       [ 5.        ,  2.        ,  0.        , ...,  8.2       ,
         2.        ,  1.        ],
       ...,
       [21.        ,  2.        ,  5.        , ..., 13.80952381,
         3.        ,  0.9048    ],
       [10.        ,  2.        ,  0.        , ..., 15.4       ,
         2.        ,  1.2       ],
       [ 3.        ,  4.        ,  2.        , ...,  5.33333333,
         1.        ,  0.6667    ]])

In [17]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [18]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [19]:
train_x

array([[[40.        ,  2.        ,  0.        , ..., 18.25      ,
          2.        ,  0.95      ]],

       [[20.        ,  1.        , 14.        , ...,  5.95      ,
          2.        ,  1.05      ]],

       [[24.        ,  1.        ,  0.        , ..., 17.54166667,
          5.        ,  1.7083    ]],

       ...,

       [[16.        ,  1.        ,  0.        , ..., 14.375     ,
          4.        ,  1.125     ]],

       [[12.        ,  2.        ,  0.        , ...,  9.5       ,
          2.        ,  1.        ]],

       [[25.        ,  1.        ,  0.        , ..., 24.04      ,
          9.        ,  2.6       ]]])

In [20]:
test_y

array([ 1,  0,  1,  1,  0,  1,  1,  2,  0,  1,  0,  1,  1,  0,  3,  1,  1,
        0,  1,  1,  6,  0,  0,  1,  1,  1,  0,  0,  2,  0,  1,  0, 14,  1,
        1,  1,  0,  1,  1,  1,  0,  1,  1,  0,  2,  1,  0,  0,  1,  2,  0,
        1,  0,  2,  1,  1,  1,  1,  0,  0,  5,  0,  0,  2,  0,  1,  1,  0,
        1,  4,  0,  1,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  1,  1,  1,
        1,  1,  1,  1,  0,  1,  0,  2,  1,  0,  0,  0,  0,  0,  0,  1,  1,
        1,  1,  1,  1,  0,  0,  0,  1,  0,  0,  1,  0,  0,  1,  1,  2,  0,
        0,  2,  1,  1,  1,  1,  0, 13,  0,  0,  4,  1,  0,  2,  0,  1,  0,
        0,  2,  0,  1,  1,  0,  8,  0,  1,  1,  1,  0,  2,  1,  0,  1,  2,
        1,  0,  1,  2,  4,  1,  0,  0,  0,  1,  2,  1,  1,  1,  2,  1,  0,
        0,  1,  1,  0,  0,  1,  1,  1,  0,  0,  2,  0,  1,  0,  1,  0,  0,
        1,  5,  0,  0,  1,  1,  1,  0,  3,  1,  0,  0,  0,  0,  1,  0,  1,
        1,  0,  0, 14,  1,  1,  0,  1,  0,  4,  0,  1,  1,  1,  0,  1,  3,
        1,  0,  1,  0,  2

In [21]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [22]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, batch_size = 10, callbacks=[tensorboard_callback])

Epoch 1/100
94/94 [==============================] - 9s 10ms/step - loss: 2.8599 - mse: 2.8599 - mae: 0.9609 - root_mean_squared_error: 1.6911 - mean_squared_logarithmic_error: 0.3857
Epoch 2/100
94/94 [==============================] - 1s 8ms/step - loss: 2.5272 - mse: 2.5272 - mae: 0.9466 - root_mean_squared_error: 1.5897 - mean_squared_logarithmic_error: 0.3265
Epoch 3/100
94/94 [==============================] - 1s 7ms/step - loss: 2.4557 - mse: 2.4557 - mae: 0.9333 - root_mean_squared_error: 1.5671 - mean_squared_logarithmic_error: 0.3155
Epoch 4/100
94/94 [==============================] - 1s 7ms/step - loss: 2.4302 - mse: 2.4302 - mae: 0.9265 - root_mean_squared_error: 1.5589 - mean_squared_logarithmic_error: 0.3141
Epoch 5/100
94/94 [==============================] - 1s 8ms/step - loss: 2.3869 - mse: 2.3869 - mae: 0.9272 - root_mean_squared_error: 1.5450 - mean_squared_logarithmic_error: 0.3098
Epoch 6/100
94/94 [==============================] - 1s 7ms/step - loss: 2.3569 - ms

94/94 [==============================] - 1s 7ms/step - loss: 1.7029 - mse: 1.7029 - mae: 0.8205 - root_mean_squared_error: 1.3049 - mean_squared_logarithmic_error: 0.2490
Epoch 89/100
94/94 [==============================] - 1s 8ms/step - loss: 1.6201 - mse: 1.6201 - mae: 0.8100 - root_mean_squared_error: 1.2728 - mean_squared_logarithmic_error: 0.2410
Epoch 90/100
94/94 [==============================] - 1s 9ms/step - loss: 1.6352 - mse: 1.6352 - mae: 0.8038 - root_mean_squared_error: 1.2787 - mean_squared_logarithmic_error: 0.2438
Epoch 91/100
94/94 [==============================] - 1s 8ms/step - loss: 1.7132 - mse: 1.7132 - mae: 0.8294 - root_mean_squared_error: 1.3089 - mean_squared_logarithmic_error: 0.2558
Epoch 92/100
94/94 [==============================] - 1s 8ms/step - loss: 1.7500 - mse: 1.7500 - mae: 0.8356 - root_mean_squared_error: 1.3229 - mean_squared_logarithmic_error: 0.2583
Epoch 93/100
94/94 [==============================] - 1s 7ms/step - loss: 1.9761 - mse: 1.976

In [23]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6007 (pid 27262), started 2:06:29 ago. (Use '!kill 27262' to kill it.)

In [24]:
score = model.evaluate(test_x, test_y)
score

14/14 [==============================] - 2s 5ms/step - loss: 2.6216 - mse: 2.6216 - mae: 0.8110 - root_mean_squared_error: 1.6191 - mean_squared_logarithmic_error: 0.2352


[2.621565103530884,
 2.621565103530884,
 0.8109859824180603,
 1.6191247701644897,
 0.235198974609375]